In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import geopandas as gpd

gpd.options.io_engine = "fiona"
import os

In [ ]:
os.getcwd()

In [ ]:
os.chdir('JSnow')

In [ ]:
os.getcwd()

In [ ]:
ls

In [ ]:
zip_path = "snow.zip"

In [ ]:
with zipfile.ZipFile(zip_path) as z:
    # Load pump locations
    df_pumps = pd.read_csv(z.open("snow6/pumps.csv"))
    
    # Load deaths aggregated by building
    df_deaths = pd.read_csv(z.open("snow2/deaths_by_bldg.csv"))

In [ ]:
#### load the streets coordinates
df_streets = pd.read_csv('streets_JSnow.csv')

In [ ]:
Average deaths < 1.0 units from Broad St pump: 2.20
Average deaths ≥ 1.0 units from Broad St pump: 1.82

In [ ]:
def transform_coords(df, x_col='COORD_X', y_col='COORD_Y'):
    """
    Transform British National Grid coordinates to HistData coordinate system.
    Uses Broad Street pump as reference point.
    """
    # Center around Broad Street pump (ID=1) and scale
    x_centered = df[x_col] - 529396.5
    y_centered = df[y_col] - 181025.0
    
    # Scale to match HistData system (roughly 1 unit = 100m)
    x_scaled = (x_centered / 100) + 13.5
    y_scaled = (y_centered / 100) + 11.5
    
    return x_scaled, y_scaled

In [ ]:
# Apply transformation to pumps
df_pumps['x_scaled'], df_pumps['y_scaled'] = transform_coords(df_pumps)

# Apply transformation to deaths
df_deaths['x_scaled'], df_deaths['y_scaled'] = transform_coords(df_deaths)

In [ ]:
# Calculate zoom area based on pump locations with padding
x_min = df_pumps['x_scaled'].min() - 1.5
x_max = df_pumps['x_scaled'].max() + 1.5
y_min = df_pumps['y_scaled'].min() - 1.5
y_max = df_pumps['y_scaled'].max() + 1.5

xlim_zoomed = (x_min, x_max)
ylim_zoomed = (y_min, y_max)

In [ ]:
plt.style.use('default')
plt.figure(figsize=(14, 10))

# Plot streets first (base layer)
for street_id in df_streets['street'].unique():
    street_data = df_streets[df_streets['street'] == street_id]
    plt.plot(street_data['x'].values, street_data['y'].values, 
             color='gray', linewidth=1, alpha=0.7)

# Plot pumps on top
plt.scatter(
    df_pumps['x_scaled'], df_pumps['y_scaled'],
    c='red', s=200, marker='*',
    label='Water Pumps', zorder=5
)

plt.title("Water Pumps and Streets in Soho (1854)", fontsize=15)
plt.xlabel("X coordinate")
plt.ylabel("Y coordinate")
plt.xlim(xlim_zoomed)
plt.ylim(ylim_zoomed)
plt.gca().invert_yaxis()
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 2. Cholera Deaths by Building with Streets
plt.figure(figsize=(14, 10))

# Plot streets first
for street_id in df_streets['street'].unique():
    street_data = df_streets[df_streets['street'] == street_id]
    plt.plot(street_data['x'].values, street_data['y'].values, 
             color='gray', linewidth=1, alpha=0.7)

# Plot deaths (size proportional to death count)
plt.scatter(
    df_deaths['x_scaled'], df_deaths['y_scaled'],
    c='black', s=(df_deaths['deaths'] + 1) * 20, marker='s',
    label='Cholera Deaths', zorder=5
)

plt.title("Cholera Deaths by Building with Streets (1854)", fontsize=15)
plt.xlabel("X coordinate")
plt.ylabel("Y coordinate")
plt.xlim(xlim_zoomed)
plt.ylim(ylim_zoomed)
plt.gca().invert_yaxis()
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 3. Combined Map: Deaths and Pumps with Streets

plt.figure(figsize=(14, 10))

# Plot streets first (base layer)
for street_id in df_streets['street'].unique():
    street_data = df_streets[df_streets['street'] == street_id]
    plt.plot(street_data['x'].values, street_data['y'].values, 
             color='gray', linewidth=1, alpha=0.7)

# Plot deaths
plt.scatter(
    df_deaths['x_scaled'], df_deaths['y_scaled'],
    c='black', s=(df_deaths['deaths'] + 1) * 20, marker='s',
    label='Cholera Deaths', zorder=5
)

# Plot pumps
plt.scatter(
    df_pumps['x_scaled'], df_pumps['y_scaled'],
    c='red', s=200, marker='*',
    label='Water Pumps', zorder=6
)

plt.title("John Snow's 1854 Cholera Map (Combined with Streets)", fontsize=15)
plt.xlabel("X coordinate")
plt.ylabel("Y coordinate")
plt.legend()
plt.xlim(xlim_zoomed)
plt.ylim(ylim_zoomed)
plt.gca().invert_yaxis()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# 4. Combined Map with Pump Labels and Streets

plt.figure(figsize=(14, 10))

# Plot streets first
for street_id in df_streets['street'].unique():
    street_data = df_streets[df_streets['street'] == street_id]
    plt.plot(street_data['x'].values, street_data['y'].values, 
             color='gray', linewidth=1, alpha=0.7)

# Plot deaths
plt.scatter(
    df_deaths['x_scaled'], df_deaths['y_scaled'],
    c='black', s=(df_deaths['deaths'] + 1) * 20, marker='s',
    label='Cholera Deaths', zorder=5
)

# Plot pumps
plt.scatter(
    df_pumps['x_scaled'], df_pumps['y_scaled'],
    c='red', s=200, marker='*',
    label='Water Pumps', zorder=6
)

# Add pump labels
for _, row in df_pumps.iterrows():
    plt.text(
        row['x_scaled'] + 0.1,
        row['y_scaled'] - 0.05,
        row['name'],
        fontsize=12,
        color='darkred',
        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7)
    )

plt.title("John Snow's 1854 Cholera Map (With Pump Labels and Streets)", fontsize=15)
plt.xlabel("X coordinate")
plt.ylabel("Y coordinate")
plt.legend()
plt.xlim(xlim_zoomed)
plt.ylim(ylim_zoomed)
plt.gca().invert_yaxis()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
print("\nDescriptive Statistics for Deaths:")
print(df_deaths['deaths'].describe())

In [ ]:
# 
# histogram with  bin alignment 
max_deaths = df_deaths['deaths'].max()
bins = np.arange(0, max_deaths + 2) - 0.5

plt.figure(figsize=(10, 6))
plt.hist(df_deaths['deaths'], bins=bins, edgecolor='black')
plt.title("Distribution of Deaths per Building", fontsize=13)
plt.xlabel("Deaths")
plt.ylabel("Frequency")
plt.xticks(range(0, max_deaths + 1))
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
### box plots

In [ ]:
df_deaths.boxplot(column='deaths') # starting with a very simple one

In [ ]:
# Box Plot of Deaths

fig, ax = plt.subplots(figsize=(10, 6))

# Create box plot with notches and smaller width
bp = ax.boxplot(df_deaths['deaths'], vert=False, patch_artist=True,
                notch=True, widths=0.3)

# Calculate mean
mean_val = df_deaths['deaths'].mean()

# Customize appearance
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][0].set_edgecolor('black')

# Add vertical dotted line for mean
ax.axvline(mean_val, color='red', linestyle=':', linewidth=2,
           label=f'Mean = {mean_val:.2f}')

# Add grid lines
ax.grid(True, axis='x', alpha=0.3, linestyle='-', linewidth=0.5)

# Add legend and labels
ax.legend(loc='upper right', fontsize=10)
ax.set_xlabel("Number of Deaths", fontsize=11)
ax.set_ylabel("")
ax.set_title("Box Plot of Deaths per Building (1854 Cholera Outbreak)", fontsize=12)
ax.set_yticks([])
plt.tight_layout()
plt.show()

In [ ]:
# Part B — Distance from each building to the nearest pump

print("\n=== Part B: Distance to Nearest Pump ===")

# Extract coordinates
death_xy = df_deaths[['x_scaled', 'y_scaled']].to_numpy()
pump_xy = df_pumps[['x_scaled', 'y_scaled']].to_numpy()

# Compute pairwise distances
from scipy.spatial import distance_matrix
dist_matrix = distance_matrix(death_xy, pump_xy)

# Add nearest pump distance to df_deaths
df_deaths['dist_to_nearest_pump'] = dist_matrix.min(axis=1)

print(df_deaths[['deaths', 'dist_to_nearest_pump']].head())

# Scatterplot: distance vs deaths
plt.figure(figsize=(10, 6))
plt.scatter(df_deaths['dist_to_nearest_pump'], df_deaths['deaths'],
            c='black', alpha=0.7)
plt.xlabel("Distance to nearest pump (scaled units)")
plt.ylabel("Deaths in building")
plt.title("Deaths vs Distance to Nearest Pump")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
print("\n=== Part C: Broad Street Pump Investigation ===")

# Identify Broad Street pump
broad = df_pumps[df_pumps['name'].str.contains("Broad", case=False)].iloc[0]

# Compute distance from each building to Broad Street pump
df_deaths['dist_to_broad'] = np.sqrt(
    (df_deaths['x_scaled'] - broad['x_scaled'])**2 +
    (df_deaths['y_scaled'] - broad['y_scaled'])**2
)

print(df_deaths[['deaths', 'dist_to_broad']].head())



In [ ]:
# Scatterplot: distance to Broad St pump vs deaths
plt.figure(figsize=(10, 6))
plt.scatter(df_deaths['dist_to_broad'], df_deaths['deaths'],
            c='darkred', alpha=0.7)
plt.xlabel("Distance to Broad Street Pump (scaled units)")
plt.ylabel("Deaths in building")
plt.title("Deaths vs Distance to Broad Street Pump")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Compare mean deaths near vs far from Broad Street pump
threshold = 1.0   # scaled units (~100m in your transform)
near = df_deaths[df_deaths['dist_to_broad'] < threshold]['deaths'].mean()
far = df_deaths[df_deaths['dist_to_broad'] >= threshold]['deaths'].mean()

print(f"\nAverage deaths < {threshold} units from Broad St pump: {near:.2f}")
print(f"Average deaths ≥ {threshold} units from Broad St pump: {far:.2f}")